After running the scrapers in Notebook 3 and manually retrieving additional data from the registries from a small number of trials, we combined this data in an excel sheet `registry_data.xlsx` detailing the data collected and where it came from. This notebook will clean up this data in a way we can use to be merged into the final dataset

In [1]:
import sys
from pathlib import Path
import os
cwd = os.getcwd()
parent = str(Path(cwd).parents[0])
sys.path.append(parent)

In [2]:
import pandas as pd
import re
import numpy as np

In [4]:
reg = pd.read_excel(parent + '/data/registry_data/registry_data_apr22.xlsx', sheet_name='Full')

ictrp = pd.read_csv(parent + '/data/cleaned_ictrp_1jul2021.csv')

In [5]:
reg.columns

Index(['trial_id', 'trial_status', 'pcd', 'scd', 'reg_results_status',
       'other_results_1', 'other_results_2', 'last_updated'],
      dtype='object')

In [6]:
ictrp.head()

,Unnamed: 0,trialid,source_register,date_registration,date_enrollement,retrospective_registration,recruitment_status,phase,study_type,countries,public_title,target_enrollment,web_address
0,0,IRCT20180223038837N1,IRCT,2018-12-08,2018-10-23,True,Recruiting,Not Applicable,Interventional,Iran,Comparative study of radiofrequency ablation &...,60,http://en.irct.ir/trial/32039
1,1,NCT04246242,ClinicalTrials.gov,2020-01-27,2020-01-25,True,Not Recruiting,Phase 4,Interventional,No Country Given,A Randomized Multicenter Controlled Clinical T...,500,https://clinicaltrials.gov/show/NCT04246242
2,2,ChiCTR2000029953,ChiCTR,2020-02-17,2020-02-01,True,Not Recruiting,Not Applicable,Observational,China,Construction and Analysis of Prognostic Predic...,400,http://www.chictr.org.cn/showproj.aspx?proj=49217
3,3,ChiCTR2000029949,ChiCTR,2020-02-16,2020-02-16,False,Not Recruiting,Not Applicable,Observational,China,A Medical Records Based Study for the Effectiv...,40,http://www.chictr.org.cn/showproj.aspx?proj=49181
4,4,ChiCTR2000029947,ChiCTR,2020-02-16,2020-03-01,False,Not Recruiting,Not Applicable,Interventional,China,A Randomized Controlled Trial for Qingyi No. 4...,200,http://www.chictr.org.cn/showproj.aspx?proj=49599


In [7]:
merged = reg.merge(ictrp[['trialid', 'web_address']], how='left', 
                   left_on='trial_id', right_on='trialid').drop('trialid', axis=1)

In [8]:
merged['pcd'] = pd.to_datetime(merged['pcd'], errors='coerce')
merged['scd'] = pd.to_datetime(merged['scd'], errors='coerce')

In [9]:
conditions = [merged.pcd.notnull(), (merged.pcd.isnull() & merged.scd.notnull()), (merged.pcd.isnull() & merged.scd.isnull())]
choices = [merged.pcd, merged.scd, None]

merged['relevant_comp_date'] = np.select(conditions, choices)
merged['relevant_comp_date'] = pd.to_datetime(merged['relevant_comp_date'], errors='coerce')

In [10]:
merged['tabular_results'] = np.where(merged.reg_results_status.isin(['Study Results', 'View results']), 1, 0)

In [11]:
filt_1 = merged.other_results_1.notnull()

filt_2 = merged.other_results_2.notnull()

In [12]:
merged['potential_other_results'] = np.where((filt_1 | filt_2), 1, 0)

In [13]:
merged.to_csv(parent + '/data/registry_data/registry_data_clean_apr22.csv')

In [14]:
merged.head(50)

,trial_id,trial_status,pcd,scd,reg_results_status,other_results_1,other_results_2,last_updated,web_address,relevant_comp_date,tabular_results,potential_other_results
0,NCT04246242,NaN,NaT,NaT,NaN,NaN,NaN,NaT,https://clinicaltrials.gov/show/NCT04246242,NaT,0,0
1,NCT04337216,NaN,NaT,NaT,NaN,NaN,NaN,NaT,https://clinicaltrials.gov/show/NCT04337216,NaT,0,0
2,NCT04343677,NaN,NaT,NaT,NaN,NaN,NaN,NaT,https://clinicaltrials.gov/show/NCT04343677,NaT,0,0
3,NCT04380948,NaN,NaT,NaT,NaN,NaN,NaN,NaT,https://clinicaltrials.gov/show/NCT04380948,NaT,0,0
4,NCT04404634,NaN,NaT,NaT,NaN,NaN,NaN,NaT,https://clinicaltrials.gov/show/NCT04404634,NaT,0,0
5,NCT04471662,NaN,NaT,NaT,NaN,NaN,NaN,NaT,https://clinicaltrials.gov/show/NCT04471662,NaT,0,0
6,NCT04244591,Completed,2020-04-13,2020-04-13,No Results Posted,"['Sultana J, Crisafulli S, Gabbay F, Lynn E, S...",NaN,2020-06-16,https://clinicaltrials.gov/show/NCT04244591,2020-04-13,0,1
7,NCT04251767,Withdrawn,2020-04-30,2020-04-30,No Results Posted,NaN,NaN,2020-03-17,https://clinicaltrials.gov/show/NCT04251767,2020-04-30,0,0
8,NCT04251871,Recruiting,2021-01-22,2021-01-22,No Results Posted,"['Akhtar S, Das JK, Ismail T, Wahid M, Saeed W...",NaN,2020-05-05,https://clinicaltrials.gov/show/NCT04251871,2021-01-22,0,1
9,NCT04252118,Recruiting,2020-12-01,2021-12-01,No Results Posted,NaN,NaN,2020-04-15,https://clinicaltrials.gov/show/NCT04252118,2020-12-01,0,0
